In [6]:
import json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
from pathlib import Path
import os
import seaborn as sns
from tqdm import tqdm
import warnings
import pickle

# Configurações
warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid", font='serif', rc={'font.serif': ['Times New Roman']})
sns.set_context("talk")

INSTITUICOES = ['uft', 'ufnt', 'ceulp', 'ifto', 'unitins', 'tocantins']

def load_network_data(institution, year):
    """Carrega os dados da rede a partir do arquivo JSON"""
    file_path = f'../results/metrics/{institution}/{institution}_{year}.json'
    with open(file_path, 'r') as f:
        data = json.load(f)
    return data

def load_real_graph(institution, year):
    """Carrega o grafo real a partir do arquivo GEXF e retorna o maior componente conexo"""
    file_path = f'../results/graphs/{institution}/graph_{institution}_{year}.gexf'
    G = nx.read_gexf(file_path)
    
    # Verifica se o grafo é conexo
    if not nx.is_connected(G):
        # Obtém o maior componente conexo
        largest_cc = max(nx.connected_components(G), key=len)
        G = G.subgraph(largest_cc).copy()
        
    return G

def save_null_models(null_stats, institution, year, save_dir):
    """Salva os modelos nulos e suas métricas em arquivos"""
    os.makedirs(f"{save_dir}/{institution}", exist_ok=True)
    
    # Salva as métricas
    metrics_path = f"{save_dir}/{institution}/null_metrics_{institution}_{year}.pkl"
    with open(metrics_path, 'wb') as f:
        pickle.dump(null_stats, f)
    
    print(f"Modelos nulos e métricas salvos em {save_dir}/{institution}/")

def load_null_models(institution, year, save_dir):
    """Carrega os modelos nulos e suas métricas de arquivos"""
    metrics_path = f"{save_dir}/{institution}/null_metrics_{institution}_{year}.pkl"
    
    if os.path.exists(metrics_path):
        with open(metrics_path, 'rb') as f:
            null_stats = pickle.load(f)
        print(f"Modelos nulos carregados de {metrics_path}")
        return null_stats
    else:
        print(f"Nenhum modelo nulo encontrado em {metrics_path}")
        return None

def generate_null_models(real_graph, num_simulations=10):
    """
    Gera modelos nulos para comparação usando apenas o maior componente conexo:
    1. Modelo Erdos-Renyi (mesmo número de nós e arestas)
    2. Modelo de Configuração (mesma distribuição de grau)
    """
    n = real_graph.number_of_nodes()
    m = real_graph.number_of_edges()
    
    # Inicializa listas para armazenar resultados
    cc_rand_list = []
    mc_rand_list = []
    cc_config_list = []
    mc_config_list = []
    
    for _ in tqdm(range(num_simulations), desc="Gerando modelos nulos"):
        # 1. Modelo Erdos-Renyi
        G_rand = nx.gnm_random_graph(n, m)
        
        # Garante que estamos trabalhando com o maior componente conexo
        if not nx.is_connected(G_rand):
            largest_cc = max(nx.connected_components(G_rand), key=len)
            G_rand = G_rand.subgraph(largest_cc).copy()
        
        # Calcula métricas
        try:
            cc_rand = nx.average_clustering(G_rand)
            mc_rand = nx.average_shortest_path_length(G_rand)
            cc_rand_list.append(cc_rand)
            mc_rand_list.append(mc_rand)
        except nx.NetworkXError as e:
            print(f"Erro no modelo Erdos-Renyi: {str(e)}")
            continue
        
        # 2. Modelo de Configuração
        degree_sequence = [d for n, d in real_graph.degree()]
        G_config = nx.configuration_model(degree_sequence)
        G_config = nx.Graph(G_config)
        G_config.remove_edges_from(nx.selfloop_edges(G_config))
        
        # Garante que estamos trabalhando com o maior componente conexo
        if not nx.is_connected(G_config):
            largest_cc = max(nx.connected_components(G_config), key=len)
            G_config = G_config.subgraph(largest_cc).copy()
        
        # Calcula métricas
        try:
            cc_config = nx.average_clustering(G_config)
            mc_config = nx.average_shortest_path_length(G_config)
            cc_config_list.append(cc_config)
            mc_config_list.append(mc_config)
        except nx.NetworkXError as e:
            print(f"Erro no modelo de Configuração: {str(e)}")
            continue
    
    # Calcula médias
    results = {
        'Erdos-Renyi': {
            'avg_clustering': np.mean(cc_rand_list) if cc_rand_list else np.nan,
            'avg_path_length': np.mean(mc_rand_list) if mc_rand_list else np.nan,
            'std_clustering': np.std(cc_rand_list) if cc_rand_list else np.nan,
            'std_path_length': np.std(mc_rand_list) if mc_rand_list else np.nan
        },
        'Configuration': {
            'avg_clustering': np.mean(cc_config_list) if cc_config_list else np.nan,
            'avg_path_length': np.mean(mc_config_list) if mc_config_list else np.nan,
            'std_clustering': np.std(cc_config_list) if cc_config_list else np.nan,
            'std_path_length': np.std(mc_config_list) if mc_config_list else np.nan
        }
    }
    
    return results

def calculate_small_worldness(real_cc, real_mc, rand_cc, rand_mc, config_cc=None, config_mc=None):
    """
    Calcula a métrica de Small-Worldness de Humphries & Gurney
    """
    if rand_cc == 0 or rand_mc == 0:
        return np.nan
    
    S = (real_cc / rand_cc) / (real_mc / rand_mc)
    
    S_config = np.nan
    if config_cc is not None and config_mc is not None and config_cc != 0 and config_mc != 0:
        S_config = (real_cc / config_cc) / (real_mc / config_mc)
    
    return {
        'Small-Worldness (vs ER)': S,
        'Small-Worldness (vs Config)': S_config
    }

def plot_comparison(institution, real_stats, null_stats, save_dir=None):
    """Plota a comparação entre a rede real e os modelos nulos"""
    models = ['Real', 'Erdos-Renyi', 'Configuration']
    cc_values = [real_stats['avg_clustering'], 
                 null_stats['Erdos-Renyi']['avg_clustering'], 
                 null_stats['Configuration']['avg_clustering']]
    cc_errors = [0, 
                 null_stats['Erdos-Renyi']['std_clustering'], 
                 null_stats['Configuration']['std_clustering']]
    
    mc_values = [real_stats['avg_path_length'], 
                 null_stats['Erdos-Renyi']['avg_path_length'], 
                 null_stats['Configuration']['avg_path_length']]
    mc_errors = [0, 
                 null_stats['Erdos-Renyi']['std_path_length'], 
                 null_stats['Configuration']['std_path_length']]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    ax1.bar(models, cc_values, yerr=cc_errors, capsize=10, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax1.set_title(f'Coeficiente de Agrupamento Médio - {institution.upper()}', fontsize=16)
    ax1.set_ylabel('Valor Médio', fontsize=14)
    ax1.tick_params(axis='x', rotation=45)
    
    ax2.bar(models, mc_values, yerr=mc_errors, capsize=10, color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax2.set_title(f'Caminho Médio Mais Curto - {institution.upper()}', fontsize=16)
    ax2.set_ylabel('Valor Médio', fontsize=14)
    ax2.tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        save_path = Path(f'{save_dir}/{institution}/') / f'{institution}_null_models_comparison.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.show()

def analyze_institution(institution, year='2024', save_dir=None, load_existing=False):
    """Realiza toda a análise para uma instituição"""
    print(f"\nAnalisando {institution.upper()}...")
    
    try:
        # Carrega os dados do JSON
        data = load_network_data(institution, year)
        
        # Carrega o grafo real (já retorna o maior componente conexo)
        G_real = load_real_graph(institution, year)
        
        # Obtém estatísticas da rede real
        real_stats = {
            'avg_clustering': data['average_clustering_coefficient'],
            'avg_path_length': data['average_shortest_path_length']
        }
        
        # Verifica se deve carregar modelos existentes
        null_stats = None
        if load_existing and save_dir:
            null_stats = load_null_models(institution, year, save_dir)
        
        # Se não carregou modelos existentes, gera novos
        if null_stats is None:
            print(f"\nGerando modelos nulos...")
            null_stats = generate_null_models(G_real)
            
            # Salva os modelos nulos gerados
            if save_dir:
                save_null_models(null_stats, institution, year, save_dir)
        
        # Calcula métrica de Small-Worldness
        print(f"\nCalculando Small-Worldness...")
        small_worldness = calculate_small_worldness(
            real_stats['avg_clustering'],
            real_stats['avg_path_length'],
            null_stats['Erdos-Renyi']['avg_clustering'],
            null_stats['Erdos-Renyi']['avg_path_length'],
            null_stats['Configuration']['avg_clustering'],
            null_stats['Configuration']['avg_path_length']
        )
        
        # Plota comparação
        plot_comparison(institution, real_stats, null_stats, save_dir)
        
        # Prepara resultados para exibição
        results = {
            'Rede Real': real_stats,
            'Modelos Nulos': null_stats,
            'Small-Worldness': small_worldness,
            'Network Stats': {
                'Número de Nós': G_real.number_of_nodes(),
                'Número de Arestas': G_real.number_of_edges(),
                'Grau Médio': sum(dict(G_real.degree()).values()) / G_real.number_of_nodes(),
                'Densidade': nx.density(G_real),
                'Componente Conexo': "Maior componente" if not nx.is_connected(load_real_graph(institution, year)) else "Grafo conexo"
            }
        }
        
        return results
    
    except Exception as e:
        print(f"Erro ao analisar {institution}: {str(e)}")
        return None

# Diretório para salvar os resultados
save_dir = "../results/img/null_models_comparison/".strip()
null_models_dir = "../results/null_models/"  # Novo diretório para salvar os modelos nulos

analysis_results = {}

# Cria os diretórios se não existirem
for directory in [save_dir, null_models_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Pergunta ao usuário se deseja carregar modelos existentes
load_existing = input("Deseja carregar modelos nulos existentes? (s/n): ").lower() == 's'

# Analisa cada instituição
for instituicao in INSTITUICOES:
    results = analyze_institution(instituicao, save_dir=save_dir, load_existing=load_existing)
    
    if results is not None:
        analysis_results[instituicao] = results
        
        # Exibe resultados
        print(f"\nResultados para {instituicao.upper()}:")
        
        print("\nEstatísticas da Rede:")
        print(f"- Número de Nós: {results['Network Stats']['Número de Nós']}")
        print(f"- Número de Arestas: {results['Network Stats']['Número de Arestas']}")
        print(f"- Grau Médio: {results['Network Stats']['Grau Médio']:.2f}")
        print(f"- Densidade: {results['Network Stats']['Densidade']:.4f}")
        print(f"- Componente Conexo: {results['Network Stats']['Componente Conexo']}")
        
        print("\nMétricas da Rede Real:")
        print(f"- Coeficiente de Agrupamento Médio: {results['Rede Real']['avg_clustering']:.4f}")
        print(f"- Caminho Médio Mais Curto: {results['Rede Real']['avg_path_length']:.4f}")
        
        print("\nComparação com Modelos Nulos:")
        print("Erdos-Renyi:")
        print(f"- Coeficiente de Agrupamento Médio: {results['Modelos Nulos']['Erdos-Renyi']['avg_clustering']:.4f} ± {results['Modelos Nulos']['Erdos-Renyi']['std_clustering']:.4f}")
        print(f"- Caminho Médio Mais Curto: {results['Modelos Nulos']['Erdos-Renyi']['avg_path_length']:.4f} ± {results['Modelos Nulos']['Erdos-Renyi']['std_path_length']:.4f}")
        
        print("\nConfiguration Model:")
        print(f"- Coeficiente de Agrupamento Médio: {results['Modelos Nulos']['Configuration']['avg_clustering']:.4f} ± {results['Modelos Nulos']['Configuration']['std_clustering']:.4f}")
        print(f"- Caminho Médio Mais Curto: {results['Modelos Nulos']['Configuration']['avg_path_length']:.4f} ± {results['Modelos Nulos']['Configuration']['std_path_length']:.4f}")
        
        print("\nSmall-Worldness:")
        print(f"- vs Erdos-Renyi: {results['Small-Worldness']['Small-Worldness (vs ER)']:.4f}")
        print(f"- vs Configuration Model: {results['Small-Worldness']['Small-Worldness (vs Config)']:.4f}")

print("\nAnálise concluída!")


Analisando UFT...

Gerando modelos nulos...


Gerando modelos nulos:  50%|████████████████████████████████████████████▌                                            | 5/10 [1:57:23<1:57:23, 1408.72s/it]


KeyboardInterrupt: 